In [146]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot') 

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

import sys

#Based on Eddie Lam's code

'''
Reusing code:
https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

Explaining result:
https://datascience.stackexchange.com/questions/65839/macro-average-and-weighted-average-meaning-in-classification-report


Test_fearMat_Clean: 4168
    Before edit: 4231 rows × 34 columns
    After edit: 4168 rows × 34 columns
    
    #removed 6 to 7
    total = 3749         (20% = 750; 80% = 2,999)
    

Train_fearMat_Clean: 16680
    Before edit: 16927 rows × 34 columns
    After edit: 16680 rows × 34 columns
    
    #removed 6 to 7
    total = 15045         (20% = 3,009; 80% = 12,036)

'''

import os


print(os.listdir(r"."))         #print out what's in the file/directory
#train = pd.read_csv(r"featMatVersionOriginal.csv")

#train = pd.read_csv(r"featMatVersion1.csv")

train = pd.read_csv(r"featMatVersion2.csv")


#/content/featMatVersion2.csv

#train = pd.read_csv(r"featMatVersion3.csv")

#train = pd.read_csv(r"featMatVersion4.csv")

#Version Orginal is the unmodified dataset from Eddie

# This Version1 uses one authentic user with a class ID of 1
# The rest have a class ID of 0

#Version 2 has sessions 6 and 7 removed

#Version 3 has sessions 6 and 7 removed and authentic user with class 1 and rest 0

#clean up the negative values for interstroke?

#compare only 2 people?

#remove columns 2 and 13?

#Make a second code for the ones with the removed columns

#Too much data to clean up

#Should try randomizing

#upload the csv to google colab
#for some reason need to do this each time
#need to find a way to store and acces from drive

#pd.read_csv('/My Drive/Collab Notebooks/featMatVersionOriginal.csv')

#Try to get data from google drive instead of manually importing it

#Maybe try removing sessions 6 and 7

['.config', '.ipynb_checkpoints', 'featMatVersionOriginal.csv', 'featMatVersion2.csv', 'featMatVersion1.csv', 'featMatVersion3.csv', 'sample_data']


In [147]:
#print
train.head()

,User_ID,Doc_ID,Inter-Stroke time,Stroke Duration,Start X,Start Y,Stop X,Stop Y,Direct end-to-end Distance,Mean resulant length,WSAD Flag,Direction of end-to-end Time,Phone_ID,20 per pairwise velocity,50 per pairwise velocity,80 per pairwise velocity,20 per pairwise acc,50 per pairwise acc,80 per pairwise acc,Median velocity at last 3pts,Largest deviation from end-to-end line,20 per dev line,50 per dev line,80 per dev line,Average Direction,Length of Trajectory,Ratio dist and length of trajectory,Average velocity,Median acceleration at first 5 pts,Mid-Stroke pressure,Mid-Stroke area covered,Mid-Stroke finger orientation,Change of finger orientation,Phone orientation
0,37,1,1.326,1.073,28.121,-37.394,32.052,-10.684,26.9980,0.92509,3,1.4247,1,8.8194,20.9990,48.686,-2222.90,-144.960,10515.00,20.5370,-2.62020,-1.940600,-1.77050,-1.252100,1.32320,28.031,0.96315,26.124,11611.000000,0.51,0.044444,0.0,0,1
1,37,1,2.934,0.704,25.602,-39.511,27.416,-24.594,15.0270,0.68201,3,1.4498,1,0.0000,18.8900,37.781,-575.53,0.000,681.86,0.0000,-0.75989,-0.697690,-0.52462,-0.446200,1.03910,16.212,0.92693,23.028,0.074442,0.47,0.088889,0.0,0,1
2,37,1,0.830,0.520,27.617,-27.214,25.602,-39.410,12.3620,0.67699,1,-1.7346,1,4.2071,16.7990,84.761,-752.52,-96.267,1556.80,13.1070,0.55393,-0.065173,0.00000,0.262660,-1.58700,14.584,0.84758,28.047,2656.100000,0.48,0.044444,0.0,0,1
3,37,1,2.214,1.728,28.625,-27.214,27.617,-31.548,4.4498,0.55335,1,-1.7993,1,0.0000,2.9645,28.894,-656.96,0.000,535.31,7.7718,-2.31740,-2.219200,-2.07530,-0.310500,-0.13008,41.967,0.10603,24.286,-367.070000,0.47,0.044444,0.0,0,1
4,37,1,0.686,0.432,27.416,-43.644,30.440,-21.671,22.1800,0.63109,3,1.4340,1,12.8810,51.2940,110.040,-2917.70,-445.010,1051.30,26.5610,-2.00710,-1.098100,-0.39071,-0.065958,1.08450,26.002,0.85303,60.189,-904.790000,0.49,0.044444,0.0,0,1


In [148]:
np.any(np.isnan(train))

False

In [149]:
np.all(np.isfinite(train))

True

In [150]:
#print out the columns
train.columns

Index(['User_ID', 'Doc_ID', 'Inter-Stroke time', 'Stroke Duration', 'Start X',
       'Start Y', 'Stop X', 'Stop Y', 'Direct end-to-end Distance',
       'Mean resulant length', 'WSAD Flag', 'Direction of end-to-end Time',
       'Phone_ID', '20 per pairwise velocity', '50 per pairwise velocity',
       '80 per pairwise velocity', '20 per pairwise acc',
       '50 per pairwise acc', '80 per pairwise acc',
       'Median velocity at last 3pts',
       'Largest deviation from end-to-end line', '20 per dev line',
       '50 per dev line', '80 per dev line', 'Average Direction',
       'Length of Trajectory', 'Ratio dist and length of trajectory',
       'Average velocity', 'Median acceleration at first 5 pts',
       'Mid-Stroke pressure', 'Mid-Stroke area covered',
       'Mid-Stroke finger orientation', 'Change of finger orientation',
       'Phone orientation'],
      dtype='object')

In [151]:
#prints the amount of users in 'User_ID'
train['User_ID'].value_counts()

2     1000
23     969
35     796
38     786
15     719
33     712
17     706
14     651
34     613
7      591
28     580
3      549
21     533
36     500
18     461
6      454
8      440
29     439
39     436
24     435
25     432
27     427
37     424
1      401
22     374
31     370
20     356
9      343
12     342
5      329
11     318
40     312
13     305
10     297
19     293
16     273
26     242
4      241
32     232
30     225
41     186
Name: User_ID, dtype: int64

In [152]:
#Choosing the columns for X
#Not using 'User_ID', 'Doc_ID', or 'Phone_ID'
X = train[['Inter-Stroke time', 'Stroke Duration', 'Start X', 'Start Y', 'Stop X', 'Stop Y', 'Direct end-to-end Distance',
        'Mean resulant length', 'WSAD Flag', 'Direction of end-to-end Time', '20 per pairwise velocity', 
        '50 per pairwise velocity', '80 per pairwise velocity', '20 per pairwise acc', '50 per pairwise acc', 
        '80 per pairwise acc', 'Median velocity at last 3pts', 'Largest deviation from end-to-end line', '20 per dev line',
        '50 per dev line', '80 per dev line', 'Average Direction', 'Length of Trajectory', 
        'Ratio dist and length of trajectory', 'Average velocity', 'Median acceleration at first 5 pts',
        'Mid-Stroke pressure', 'Mid-Stroke area covered', 'Mid-Stroke finger orientation', 'Change of finger orientation',
        'Phone orientation']].values

In [153]:
#Choosing the lables for y
y = train['User_ID'].values

In [154]:
#Train and test split
#80% train, 20% test
#splitting the data into test and train dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [155]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [156]:
#Training and Prediction
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)         #value for k
classifier.fit(X_train, y_train)

#The k value is guessed at first and then changed depending on the data
#5 is used by convention but is usually a guess
#Might want to change to 7 as when I did for K values up to 100 7 seemed smaller
#on the smaller graph though 5 seems to be the best answer
#The graph fluctuates with each run
# 6 and 5 seem to be good choices


#For the 0 and 1 dataset the better k values are 4, 7 and 9
#Might want to make 2 seperate files

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [157]:
#Make prediction on the test data
y_pred = classifier.predict(X_test)

In [158]:
#Evaluating the Algorithm
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 58   1   0   3   0   0   6   0   0   0   0   0   0   0   3   0   0   0
    0   1   0   3   1   1   1   4   0   3   0   0   0   0   0   2   0   0
    0   0   1   0   0]
 [  0 172   0   1   0   0   0   1   4   0   0   0   0   0   0   0   0   0
    1   0   0   0   3   0   0   1   1   2   0   0   0   0   0   0   0   0
    0   2   2   0   0]
 [  0   0  53   0   4   4   0   3   1   2   2   2   0   0   0   1  13   1
    1   1   3   0   0   0   0   0   5   0   0   0   0   0   1   0   9   0
    1   0   1   0   0]
 [  5   1   0  35   1   0   1   0   0   0   0   0   0   0   0   0   0   0
    0   2   0   0   0   0   1   1   0   2   0   0   0   0   0   2   0   0
    0   2   0   0   2]
 [  1   0   5   1  30   2   0   5   1   0   0   1   0   0   0   0   0   0
    3   0   0   0   1   0   0   0   1   0   0   0   1   1   0   0   0   0
    0   0   0   6   0]
 [  0   2   3   0   0  67   0   3   0   0   0   1   1   0   0   2   1   0
    1   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0   1  

In [159]:
#prints the entire confusion matrix
#however, it doesnn't fit the entire row and loops around
np.set_printoptions(threshold=sys.maxsize)
resultCM = (confusion_matrix(y_test, y_pred))
print(resultCM)

[[ 58   1   0   3   0   0   6   0   0   0   0   0   0   0   3   0   0   0
    0   1   0   3   1   1   1   4   0   3   0   0   0   0   0   2   0   0
    0   0   1   0   0]
 [  0 172   0   1   0   0   0   1   4   0   0   0   0   0   0   0   0   0
    1   0   0   0   3   0   0   1   1   2   0   0   0   0   0   0   0   0
    0   2   2   0   0]
 [  0   0  53   0   4   4   0   3   1   2   2   2   0   0   0   1  13   1
    1   1   3   0   0   0   0   0   5   0   0   0   0   0   1   0   9   0
    1   0   1   0   0]
 [  5   1   0  35   1   0   1   0   0   0   0   0   0   0   0   0   0   0
    0   2   0   0   0   0   1   1   0   2   0   0   0   0   0   2   0   0
    0   2   0   0   2]
 [  1   0   5   1  30   2   0   5   1   0   0   1   0   0   0   0   0   0
    3   0   0   0   1   0   0   0   1   0   0   0   1   1   0   0   0   0
    0   0   0   6   0]
 [  0   2   3   0   0  67   0   3   0   0   0   1   1   0   0   2   1   0
    1   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0   1  

In [160]:
resultCR = (classification_report(y_test, y_pred))
print(resultCR)

              precision    recall  f1-score   support

           1       0.63      0.66      0.64        88
           2       0.75      0.91      0.82       190
           3       0.56      0.49      0.52       108
           4       0.69      0.64      0.66        55
           5       0.43      0.51      0.47        59
           6       0.58      0.80      0.67        84
           7       0.63      0.60      0.62       111
           8       0.56      0.67      0.61        91
           9       0.64      0.87      0.73        67
          10       0.70      0.58      0.64        65
          11       0.56      0.74      0.63        54
          12       0.68      0.80      0.73        64
          13       0.55      0.46      0.50        67
          14       0.62      0.79      0.70       134
          15       0.69      0.76      0.72       139
          16       0.52      0.52      0.52        58
          17       0.64      0.66      0.65       137
          18       0.80    

In [161]:
#error = []

# Calculating error for K values between 1 and 40
#Takes a long time to load
# Commented out as it takes a long time

#for i in range(1, 40):
    #knn = KNeighborsClassifier(n_neighbors=i)
    #knn.fit(X_train, y_train)
    #pred_i = knn.predict(X_test)
    #error.append(np.mean(pred_i != y_test))

In [162]:
#Commented out as it relies on the above

#plt.figure(figsize=(12, 6))
#plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o',
#         markerfacecolor='blue', markersize=10)
#plt.title('Error Rate K Value')
#plt.xlabel('K Value')
#plt.ylabel('Mean Error')